##### Importacao de bibliotecas e banco de dados

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
base_path = Path("database")
base_airbnb = pd.DataFrame()
for file in base_path.iterdir():
    if file.name.endswith(".csv"):
        month_df = pd.read_csv(base_path / file.name)
        month_df['last_scraped'] = pd.to_datetime(month_df['last_scraped'])
        month_df['year'] = month_df['last_scraped'].dt.year
        month_df['month'] = month_df['last_scraped'].dt.month
        base_airbnb = pd.concat([base_airbnb, month_df], ignore_index=True)


##### Limpeza do dataset

Como tempo muitas colunas no nosso dataset, o modelo fica mais lento.

Uma análise será feita para retirar as colunas que não serão necessárias para o nosso modelo de previsao.

colunas que serão excluídas:

- IDs, links e informações nao relevantes ao modelo.
- criada colunas separadas para ano e mes de extracao (mais util para a análise), excluindo a coluna com a data completa
- Colunas preenchidas com texto livre (nao será utilizado nenhuma análise de palavras, ou LLM, focando-se um modelo mais simples)
- colunas com todos (ou proximo a isso) são iguais
- precos que nao sejam diarios
- informacoes fiscais exigidas (análise busca verificar características do imovel)

In [ ]:
print(list(base_airbnb.columns))

In [ ]:
columns = [
    "host_response_time",
    "host_response_rate",
    "host_is_superhost",
    "host_listings_count",
    "latitude",
    "longitude",
    "property_type",
    "room_type",
    "accommodates",
    "bathrooms",
    "bedrooms",
    "beds",
    "bed_type",
    "amenities",
    "price",
    "security_deposit",
    "cleaning_fee",
    "guests_included",
    "extra_people",
    "minimum_nights",
    "maximum_nights",
    "number_of_reviews",
    "review_scores_rating",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_checkin",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_value",
    "instant_bookable",
    "is_business_travel_ready",
    "cancellation_policy",
    "year",
    "month",
]
base_airbnb = base_airbnb.loc[:, columns]
display(base_airbnb)

#### Tratando dados Faltantes

Visualizando os dados, percebemos que existem uma grande disparidade em dados faltantes.
- as colunas comm mais de 300 mil valores nulos foram excluidas da análise
- Para as demais colunas, como temos mais de 900 mil linhas, vamos simplesmente exlcuir com os dados nulos

In [ ]:
print(base_airbnb.isnull().sum())
for column in base_airbnb:
    if base_airbnb[column].isnull().sum() > 300000:
        base_airbnb = base_airbnb.drop(column, axis=1)
print(base_airbnb.shape)

In [ ]:
base_airbnb = base_airbnb.dropna()
print(base_airbnb.isnull().sum())
print(base_airbnb.shape)

In [ ]:
print(base_airbnb.dtypes)
print("-"*60)
print(base_airbnb.iloc[0])

Como preço e extra_people estão sendo reconhecidos como objeto ao ines de float, tempo de mudar o tipo de variável da coluna

In [ ]:
# alterar price, extra_people
base_airbnb["price"] = base_airbnb["price"].str.replace("$", "").str.replace(",", "")
base_airbnb["price"] = base_airbnb["price"].astype(np.float32, copy=False)
base_airbnb["extra_people"] = base_airbnb["extra_people"].str.replace("$", "").str.replace(",", "")
base_airbnb["extra_people"] = base_airbnb["extra_people"].astype(np.float32, copy=False)
print(base_airbnb.dtypes)